In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
# 토큰화 후 저장한 csv 파일 불러오기 

# df = pd.read_csv(r'D:\python_project\chaekchecklab\data\final_book3_token.csv',usecols=range(1, 16))
df = pd.read_csv(r'D:\python_project\chaekchecklab\data\final_book3_token.csv', index_col=0)
df

,title,full_name,book_url,publisher,author_list,intro,category,date,pages,isbn,저자,역자,기타,processed_intro,tokenize_intro
0,생성형 AI시대 산업별 기술혁신과 서비스사례 및 법적이슈 현안분석,생성형 AI시대 산업별 기술혁신과 서비스사례 및 법적이슈 현안분석 [ 양장 ],https://www.yes24.com/Product/Goods/124435593,지식산업정보원,"[['R&D정보센터', '244360']]",본원 R&D정보센터에서는 글로벌 거대 기업들의 약진이 거세지는 생성형 인공지능 시장...,1001025007004004,2025-04-01,630쪽,9788965960270,R&D정보센터저,NaN,NaN,본원 rd정보센터에서는 글로벌 거대 기업들의 약진이 거세지는 생성형 인공지능 시장에...,본원 rd 정보 센터 에서는 글로벌 거대 기업 들 약진 거세지다 생 성형 인공 지능...
1,2024 대박 날 바이오 다크호스,2024 대박 날 바이오 다크호스,https://www.yes24.com/Product/Goods/123183541,이데일리,"[['팜이데일리 특별취재팀', '405108']]",『2024 대박 날 바이오 다크호스』와 함께미지의 길을 성공의 길로!『2024 대박...,1001025008005,2024-10-27,276쪽,9791198137159,팜이데일리 특별취재팀,NaN,NaN,<num> 대박 날 바이오 다크호스와 함께미지의 길을 성공의 길로<num> 대박 날...,< num > 대박 날 바이오 다크호스 와 함께 미지 길 성공 길 로 < num >...
2,누구나 쉽게 배우는 실전 마케팅 전략,누구나 쉽게 배우는 실전 마케팅 전략 사례로 배우는 핵심 스킬,https://www.yes24.com/Product/Goods/133819969,다온길,"[['백미르', '424835']]",마케팅으로 세상을 바꾼다최고의 브랜드가 되는 법 : 마케팅 전략과 사례로 배우다마케...,"1001025009001, 1001025009004, 1001025011001",2024-10-25,224쪽,9791162751060,백미르,NaN,NaN,마케팅으로 세상을 바꾼다최고의 브랜드가 되는 법 마케팅 전략과 사례로 배우다마케팅...,마케팅 으로 세상 바꾸다 최고 브랜드 되다 법 마케팅 전략 과 사례 로 배우다 마케...
3,도쿄 트렌드 인사이트 2025,도쿄 트렌드 인사이트 2025,https://www.yes24.com/Product/Goods/133966202,원앤원북스,"[['정희선', '232242']]","저성장, 인구 감소의 시대어떻게 소비자를 설득하고 새로운 시장을 만들 것인가 지난해...","1001025007004004, 1001025009005",2024-10-24,336쪽,9791185654546,정희선,NaN,NaN,저성장 인구 감소의 시대어떻게 소비자를 설득하고 새로운 시장을 만들 것인가 지난해 ...,저 성장 인구 감소 시 대 어떻 게 소비자 설득 하고 새롭다 시장 만들다 것 인가 ...
4,사례로 알아보는 돈 버는 부동산 경매,사례로 알아보는 돈 버는 부동산 경매 특수물건 경매 전문 변호사의 경매 비법,https://www.yes24.com/Product/Goods/133821086,새로운제안,"[['주희진', '443331']]",특수물건 부동산 경매를 통해 큰 수익을 올리고 싶은 독자들을 위해 다양한 성공사례와...,1001025010003,2024-10-21,320쪽,9788963462608,주희진,NaN,NaN,특수물건 부동산 경매를 통해 큰 수익을 올리고 싶은 독자들을 위해 다양한 성공사례와...,특수 물건 부동산 경매 통해 크다 수익 올리다 싶다 독자 들 위해 다양하다 성공사례...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56927,연봉 말고 5000만 원 더 벌기 + 2024 벤꾸리 가계부 다이어리,연봉 말고 5000만 원 더 벌기 + 2024 벤꾸리 가계부 다이어리 [ 전2권...,https://www.yes24.com/Product/Goods/123271846,시원북스,"[['강희연', '405528']]",이 상품은 YES24에서 구성한 상품입니다.(낱개 반품 불가).[도서] 연봉 말고 ...,1001025010004,NaN,600쪽,9788984054813,강희연,NaN,NaN,이 상품은 yes<num>에서 구성한 상품입니다낱개 반품 불가도서 연봉 말고 <nu...,상품 yes < num > 에서 구성 한 상품 이다 낱개 반품 불가 도서 연봉 말고...
56928,"중국주식, 저평가된 강한 기업에 투자하라 + 지금 중국 주식 천만원이면 10년 후 ...","중국주식, 저평가된 강한 기업에 투자하라 + 지금 중국 주식 천만원이면 10년 후 ...",https://www.yes24.com/Product/Goods/15114743,YES24묶음상품-도서,"[['정순필'], ['정영재']]",* 지금 중국 주식 천만원이면 10년 후 강남 아파트를 산다\r\n\r\n지금 현재...,1001025010002,NaN,744쪽,9788958430971,"정순필,정영재",NaN,NaN,지금 중국 주식 천만원이면 <num>년 후 강남 아파트를 산다 지금 현재 중국 주...,지금 중국 주식 천만원 면 < num > 년 후 강남 아파트 산다 지금 현재 중국 ...
56929,직원이라면 어떻게 일해야 하는가,직원이라면 어떻게 일해야 하는가,https://www.yes24.com/Product/Goods/16025156,원앤원북스,[['김경준']],직장인을 위한 조직생활 교과서조직사회는 사회인이라면 누구나 한 번은 거쳐야 할 인생...,1001025001006,NaN,300쪽,9788950907969,김경준,NaN,NaN,직장인을 위한 조직생활 교과서조직사회는 사회인이라면 누구나 한 번은 거쳐야 할 인생...,직장인 위 한 조직 생활 교과서 조직 사회 사회인 라면 누구 나 번은 거치다 인생 ...
56930,충전수업 세트,충전수업 세트 쩐의 흐름 + 부의 증식 [ 전2권 ],https://www.yes24.com/Product/Goods/59682596,아라크네,"[['양보석', '214044']]",이 상품은 YES24에서 구성한 상품입니다.(낱개 반품 불가).[도서] 충전수업 :...,1001025010004,NaN,쪽수확인중,9788943901905,양보석,NaN,NaN,이 상품은 yes<num>에서 구성한 상품입니다낱개 반품 불가도서 충전수업 쩐의 ...,상품 yes < num > 에서 구성 한 상품 이다 낱개 반품 불가 도서 충전 수업...


In [5]:
def get_recommendations(book_title, df, k=10):
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['tokenize_intro'])

    # 쿼리 책에 대하여, 모든 책들과의 코사인 유사도 계산
    idx = df[df['title'] == book_title].index[0]
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # 유사도 점수를 기준으로 내림차순 정렬 -> Top K권
    similar_books_idx = cosine_sim.argsort()[-(k+1):][::-1]

    # 추천 결과와 유사도 점수를 함께 반환
    recommended_books = df.iloc[similar_books_idx].copy()
    recommended_books['similarity'] = cosine_sim[similar_books_idx]

    return recommended_books[['title', 'similarity']]

In [6]:
q_book = "회계 천재가 된 홍대리"
df_top_k = get_recommendations(q_book, df, 2000)
df_top_k

,title,similarity
46468,회계 천재가 된 홍대리,1.000000
18640,회계 천재가 된 홍대리 1,0.697197
18641,회계 천재가 된 홍대리 2,0.651016
18638,회계 천재가 된 홍대리 세트 (전5권),0.645619
18639,회계 천재가 된 홍대리 3,0.645509
...,...,...
53234,주식으로 돈 버는 사람들의 투자원칙,0.054958
27276,한 걸음만 더,0.054942
22066,최신 세무&회계 용어사전 2016,0.054890
43057,서비스 꽃은 세일즈다,0.054868


In [7]:
def get_mmr_recommendations(df_top_k, cosine_sim, df, alpha=0.5, num_recommendations=5):
    # 책 제목에 대한 인덱스 가져오기
    # idx = df[df['title'] == book_title].index[0]
    idx = 0

    # 관련성 점수 (코사인 유사도)
    scores = cosine_sim[idx]

    # MMR을 위한 초기 후보 목록 생성
    recommended_indices = set()
    ranked_indices = np.argsort(scores)[::-1]  # 내림차순 정렬
    # print(ranked_indices)

    for i in ranked_indices[1:]:
        if len(recommended_indices) >= num_recommendations:
            break
        
        if i not in recommended_indices:
            # MMR 계산
            relevance_score = scores[i]
            diversity_score = sum(cosine_sim[i][j] for j in recommended_indices) if recommended_indices else 0
            mmr_score = alpha * relevance_score - (1 - alpha) * diversity_score

            # print(i, mmr_score, relevance_score, diversity_score)
            # MMR 점수가 높은 항목 선택
            if mmr_score > 0:
                recommended_indices.add(i)

    # 추천 결과 반환
    recommended_books = df_top_k.iloc[list(recommended_indices)].copy()
    recommended_books['similarity'] = [scores[i] for i in recommended_indices]

    return recommended_books[['title', 'similarity']]

In [8]:
tfidf_top_k = tfidf_matrix[df_top_k.index, :].toarray()
# 코사인 유사도 행렬 계산
cosine_sim = cosine_similarity(tfidf_top_k, tfidf_top_k)
# 추천할 책 제목
book_title = '회계 천재가 된 홍대리'

# 추천 결과 가져오기
recommendations = get_mmr_recommendations(df_top_k, cosine_sim, df, alpha=0.75, num_recommendations=10)
recommendations

NameError: name 'tfidf_matrix' is not defined

In [152]:
df_top_k.head(11)

,title,similarity
46468,회계 천재가 된 홍대리,1.000000
18640,회계 천재가 된 홍대리 1,0.697197
18641,회계 천재가 된 홍대리 2,0.651016
18638,회계 천재가 된 홍대리 세트 (전5권),0.645619
18639,회계 천재가 된 홍대리 3,0.645509
49725,대한민국에서 제일 쉬운 회계,0.620941
15552,회계 공부는 난생 처음입니다만,0.615539
20097,하버드 MBA의 회계수업,0.613067
18642,회계 천재가 된 홍대리 5,0.600395
55264,회계원리 @ 새로운 이해,0.596487


In [84]:
for r in cosine_sim:
    print(*r)

1.0000000000000004 0.4629316705766049 0.4395473208778503 0.4010119504458939 0.3593077751295041 0.343942282940204 0.3431446391173844 0.33232897425227326 0.325739558773271 0.32471951251958087 0.31886811866525683
0.4629316705766049 1.0000000000000002 0.7185136953209786 0.6526786056050853 0.6187975284727144 0.5317462860347278 0.5702948345886363 0.5456924565608781 0.5266324907407122 0.49438622349328243 0.5015280430098822
0.4395473208778503 0.7185136953209786 1.0 0.650958335409558 0.6167063754287319 0.45871089056014736 0.5723187377936878 0.6269062065407159 0.6896430125048009 0.5603171792733277 0.6082069717616474
0.4010119504458939 0.6526786056050853 0.650958335409558 1.0000000000000002 0.5176656783809174 0.40316014020276997 0.49169847635097774 0.45832555075696046 0.4811084968649681 0.45225011795971465 0.458353596562167
0.3593077751295041 0.6187975284727144 0.6167063754287319 0.5176656783809174 1.0 0.3969549951975335 0.45062776226118567 0.4691869434054957 0.5050970994213917 0.4255667174964667

In [ ]:
def get_mmr_recommendations(seratch_book_title, cosine_sim,  df, alpha=0.5, num_recommendations=5): 
    # 책 제목 인덱스 가져오기 
    idx = df[df['title'] == search_book_title].index[0]
    # 관련성 점수 
    scores = cosine_sim[idx]

    recommended_indices = set()
    ranked_indices = np.argsort(scores)[::-1]

    

In [ ]:
# 여러권 
# 냠_책소개 fasttext : 

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['tokenize_intro'])

for book_title in book_titles: 



In [ ]:
########### 여러권 ##############
def get_mmr_recommendations(df_top_k, cosine_sim, df, alpha=0.5, num_recommendations=5):
    # 책 제목에 대한 인덱스 가져오기
    # idx = df[df['title'] == book_title].index[0]
    idx = 0

    # 관련성 점수 (코사인 유사도)
    scores = cosine_sim[idx]

    # MMR을 위한 초기 후보 목록 생성
    recommended_indices = set()
    ranked_indices = np.argsort(scores)[::-1]  # 내림차순 정렬
    # print(ranked_indices)

    for i in ranked_indices[1:]:
        if len(recommended_indices) >= num_recommendations:
            break
        
        if i not in recommended_indices:
            # MMR 계산
            relevance_score = scores[i]
            diversity_score = sum(cosine_sim[i][j] for j in recommended_indices) if recommended_indices else 0
            mmr_score = alpha * relevance_score - (1 - alpha) * diversity_score

            # print(i, mmr_score, relevance_score, diversity_score)
            # MMR 점수가 높은 항목 선택
            if mmr_score > 0:
                recommended_indices.add(i)

    # 추천 결과 반환
    recommended_books = df_top_k.iloc[list(recommended_indices)].copy()
    recommended_books['similarity'] = [scores[i] for i in recommended_indices]

    return recommended_books[['title', 'similarity']]

In [2]:

def get_mmr_recommendations(book_titles, df_top_k, cosine_sim, df, alpha=0.5, num_recommendations=5):
    recommended_indices = set()

    for book_title in book_titles:
        # 책 제목에 대한 인덱스 가져오기
        try:
            idx = df[df['title'] == book_title].index[0]
        except IndexError:
            print(f"Book title '{book_title}' not found in the dataset.")
            continue

        # 관련성 점수 (코사인 유사도)
        scores = cosine_sim[idx]

        # MMR을 위한 초기 후보 목록 생성
        ranked_indices = np.argsort(scores)[::-1]  # 내림차순 정렬

        for i in ranked_indices[1:]:
            if len(recommended_indices) >= num_recommendations:
                break

            if i not in recommended_indices:
                # MMR 계산
                relevance_score = scores[i]
                diversity_score = sum(cosine_sim[i][j] for j in recommended_indices) if recommended_indices else 0
                mmr_score = alpha * relevance_score - (1 - alpha) * diversity_score

                # MMR 점수가 높은 항목 선택
                if mmr_score > 0:
                    recommended_indices.add(i)

    # 추천 결과 반환
    recommended_books = df_top_k.iloc[list(recommended_indices)].copy()
    recommended_books['similarity'] = [cosine_sim[idx][i] for i in recommended_indices]

    return recommended_books[['title', 'similarity']]

# 사용 예시
book_titles = ['부의 시나리오', '레버리지']  # 입력받은 여러 권의 책 제목
recommendations = get_mmr_recommendations(book_titles, df_top_k, cosine_sim, df, alpha=0.5, num_recommendations=5)
print(recommendations)


NameError: name 'df_top_k' is not defined

In [13]:
# 대규모 텍스트 데이터 처리에 유용한 gensim을 이용해 tfidf 
import numpy as np
import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.matutils import sparse2full
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(book_titles, df, k=10):
    # 토큰화된 문서 리스트 생성
    documents = df['tokenize_intro'].apply(lambda x: x.split()).tolist()
    
    # Gensim 사전 및 코퍼스 생성
    dictionary = Dictionary(documents)
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    # TF-IDF 모델 생성
    tfidf = TfidfModel(corpus)
    tfidf_corpus = tfidf[corpus]

    # 입력된 책들의 인덱스 구하기
    indices = []
    for title in book_titles:
        if title in df['title'].values:
            indices.append(df[df['title'] == title].index[0])
        else:
            print(f"Warning: Book title '{title}' not found in the dataset.")
    
    # 인덱스가 존재하지 않는 경우 예외 처리
    if not indices:
        raise ValueError("None of the input book titles were found in the dataset.")

    # 입력된 책들의 TF-IDF 벡터 평균 구하기
    input_vectors = [sparse2full(tfidf_corpus[idx], len(dictionary)) for idx in indices]
    mean_vector = np.mean(input_vectors, axis=0).reshape(1, -1)

    # 전체 책들과의 코사인 유사도 계산
    tfidf_matrix = np.array([sparse2full(doc, len(dictionary)) for doc in tfidf_corpus])
    cosine_sim = cosine_similarity(mean_vector, tfidf_matrix).flatten()

    # 유사도 점수를 기준으로 내림차순 정렬
    similar_books_idx = cosine_sim.argsort()[-(k+1):][::-1]  # 입력된 책들 제외 상위 k권

    # 입력된 책들이 유사도 계산 결과에서 제외되도록 필터링
    similar_books_idx = [idx for idx in similar_books_idx if idx not in indices][:k]

    # 추천 결과와 유사도 점수를 함께 반환
    recommended_books = df.iloc[similar_books_idx].copy()
    recommended_books['similarity'] = cosine_sim[similar_books_idx]

    return recommended_books[['title', 'similarity']]

# 예제 사용
# book_titles = ['book1', 'book2', 'book3', 'book4', 'book5']
# recommendations = get_recommendations(book_titles, df)
# print(recommendations)



In [15]:
# 예제 사용
book_titles = ['부의 시나리오', '레버리지', '하버드 MBA의 회계수업', '프로슈머 마케팅', '서비스 꽃은 세일즈다']
recommendations = get_recommendations(book_titles, df)
print(recommendations)

MemoryError: Unable to allocate 13.4 GiB for an array with shape (56932, 62955) and data type float32

In [ ]:
# 입력 책 권수를 5권으로 늘리는 코드 - 벡터화 1. 한권의 intro 의 tfidf 수치가 하나 나오나? 아니면 단어별로 나오나? 단어별-> 책 한권으로 결정되는 
# 흐름이 어떻게 되는지 모르겠다. 

In [ ]:
# 벡터화 2. 5권의 intro 글을 하나로 취급하고 그 안에서 tfidf 를 해서 하나의 벡터로 만든다. 

In [ ]:
# 책 별로 키워드를 뽑아서 저장해두는 것은? 칼럼 만들어서. 

In [ ]:
# 유사도 계산 